In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
  series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
  return series[..., np.newaxis].astype(np.float32)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [4]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

model = keras.models.Sequential([
  keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                      input_shape=[None, 1]),
  keras.layers.GRU(20, return_sequences=True),
  keras.layers.GRU(20, return_sequences=True),
  keras.layers.TimeDistributed(keras.layers.Dense(10))
])

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])

history = model.fit(X_train, Y_train[:, 3::2], epochs=20, 
                    validation_data=(X_valid, Y_valid[:, 3::2]))

Epoch 1/20
219/219 [==============================] - 5s 23ms/step - loss: 0.0687 - last_time_step_mse: 0.0607 - val_loss: 0.0468 - val_last_time_step_mse: 0.0381
Epoch 2/20
219/219 [==============================] - 4s 20ms/step - loss: 0.0401 - last_time_step_mse: 0.0313 - val_loss: 0.0368 - val_last_time_step_mse: 0.0292
Epoch 3/20
219/219 [==============================] - 5s 22ms/step - loss: 0.0322 - last_time_step_mse: 0.0231 - val_loss: 0.0299 - val_last_time_step_mse: 0.0198
Epoch 4/20
219/219 [==============================] - 5s 21ms/step - loss: 0.0279 - last_time_step_mse: 0.0176 - val_loss: 0.0264 - val_last_time_step_mse: 0.0155
Epoch 5/20
219/219 [==============================] - 4s 20ms/step - loss: 0.0252 - last_time_step_mse: 0.0141 - val_loss: 0.0242 - val_last_time_step_mse: 0.0126
Epoch 6/20
219/219 [==============================] - 5s 21ms/step - loss: 0.0236 - last_time_step_mse: 0.0125 - val_loss: 0.0237 - val_last_time_step_mse: 0.0125
Epoch 7/20
219/219 [==